In [ ]:
import cogent3
from cogent3 import load_aligned_seqs
from cogent3 import get_app
from phylim.apps import phylim


@cogent3.app.composable.define_app
def renamer(seqs: cogent3.app.typing.UnalignedSeqsType) -> cogent3.app.typing.UnalignedSeqsType:
    """
    A function to rename sequences in a dataset.
    """

    name_map = {
        "homo_sapiens": "Human",
        "pan_troglodytes": "Chimpanzee",
        "gorilla_gorilla": "Gorilla"
    }

    seqs = seqs.rename_seqs(lambda x: name_map.get(x.split("-")[0], x))

    return seqs.take_seqs(list(name_map.values()))

In [ ]:
folder_in = paths.DATA_APES114 + 'intergenic/'
in_dstore = cogent3.open_data_store(folder_in, suffix='fa', mode='r')

print(in_dstore.describe)

In [ ]:
#perform a codon alignment and remove stop codons
loader = get_app("load_aligned", moltype="dna")
omit_degs = get_app("omit_degenerates", moltype="dna")
#rename app makes sure that each alignment contains only one alignment per species
#some alignment files include duplicates. We take randomly a sequence per species and discard the others
rename = renamer()
concat = get_app("concat", moltype="dna")

app = loader + omit_degs + rename
results_allpos = [r for r in app.as_completed(in_dstore[:], parallel=True) if r]
alpos_alns = concat(results_allpos)
alpos_alns.source = "intergenicAR_test_alignments"
alpos_alns


   0%|          |00:00<?

NotCompleted(type=ERROR, origin=concat, source="intergenicAR_test_alignments", message="empty data")

In [3]:
#Get distances of concatenated sequences 
#time_het=max sets a 36 params substitution model
sm = get_app("model", "GN", time_het="max", optimise_motif_probs=True, show_progress=True)
result_intergenic = sm(alpos_alns)

checker = get_app("phylim")
checked = checker(result_intergenic)
lengthstree = result_intergenic.lf.get_lengths_as_ens()
print("Is model identifiable? " + str(checked.is_identifiable) + "\n")
print("Model results : \n" + str(result_intergenic.lf) + "\n")
print("Distances: " + str(lengthstree))

   0%|          |00:00<?

   0%|          |00:00<?

Is model identifiable? True

Model results : 
GN
log-likelihood = -3788072.9446
number of free parameters = 39
edge          parent    length     A>C     A>G     A>T     C>A     C>G
----------------------------------------------------------------------
Human         root        0.01    0.96    3.87    0.80    1.36    1.46
Gorilla       root        0.01    0.92    4.24    0.66    1.21    1.39
Chimpanzee    root        0.01    1.05    3.51    0.88    1.19    1.41
----------------------------------------------------------------------

continued: 
 C>T     G>A     G>C     G>T     T>A     T>C
--------------------------------------------
6.91    6.94    1.40    1.33    0.71    3.67
6.55    6.52    1.39    1.30    0.73    3.95
6.60    6.56    1.46    1.23    0.81    3.54
--------------------------------------------

   A       C       G       T
----------------------------
0.26    0.23    0.23    0.27
----------------------------

Distances: {'Human': np.float64(0.007637909897239089), 'Gorill